In [5]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [6]:
from ETL.extract.extract import Extract
extract_class = Extract()

ModuleNotFoundError: No module named 'ETL'

Data sales_first

In [7]:
path_file = 'Data_sales_source/ElectronicsProductsPricingData.csv'


In [8]:
df_sales = pd.read_csv(path_file)

FileNotFoundError: [Errno 2] No such file or directory: 'Data_sales_source/ElectronicsProductsPricingData.csv'

In [ ]:
df_sales.head(3)

,id,prices.amountMax,prices.amountMin,prices.availability,prices.condition,prices.currency,prices.dateSeen,prices.isSale,prices.merchant,prices.shipping,...,name,primaryCategories,sourceURLs,upc,weight,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30
0,AVphzgbJLJeJML43fA0o,104.99,104.99,Yes,New,USD,"2017-03-30T06:00:00Z,2017-03-10T22:00:00Z,2017...",False,Bestbuy.com,NaN,...,Sanus VLF410B1 10-Inch Super Slim Full-Motion ...,Electronics,https://www.amazon.com/Sanus-VLF410B1-10-Inch-...,7.94E+11,32.8 pounds,NaN,NaN,NaN,NaN,NaN
1,AVpgMuGwLJeJML43KY_c,69.00,64.99,In Stock,New,USD,2017-12-14T06:00:00Z,True,Walmart.com,Expedited,...,Boytone - 2500W 2.1-Ch. Home Theater System - ...,Electronics,http://reviews.bestbuy.com/3545/4784804/review...,6.42E+11,14 pounds,NaN,NaN,NaN,NaN,NaN
2,AVpgMuGwLJeJML43KY_c,69.00,69.00,In Stock,New,USD,2017-09-08T05:00:00Z,False,Walmart.com,Expedited,...,Boytone - 2500W 2.1-Ch. Home Theater System - ...,Electronics,http://reviews.bestbuy.com/3545/4784804/review...,6.42E+11,14 pounds,NaN,NaN,NaN,NaN,NaN


In [9]:
df_sales.info()

NameError: name 'df_sales' is not defined

In [10]:
# cheking null data

df = df_sales.isna().sum().reset_index()
df.columns = ['col', "null_data"]

cond = df['null_data'] > 0
df_null = df[cond]
df_null['null_percentage'] = (df['null_data'] / len(df_sales) )* 100
df_null

NameError: name 'df_sales' is not defined

In [11]:
# comple data null

data = df_sales[['prices.amountMin', 'prices.amountMax', 'prices.shipping', 'manufacturer', 'name']]


NameError: name 'df_sales' is not defined

In [12]:
data_price_shipping = data[['prices.amountMin', 'prices.amountMax', 'prices.shipping']]
data_price_shipping = data_price_shipping[data_price_shipping['prices.shipping'].isna()]
df_High_45 = data_price_shipping[data_price_shipping['prices.amountMax'] > 45]
df_Low_45 = data_price_shipping[data_price_shipping['prices.amountMax'] < 45]

NameError: name 'data' is not defined

In [13]:
df_Low_45[df_Low_45['prices.amountMin'] < 35]

NameError: name 'df_Low_45' is not defined

untuk data price shipping dilakukan mapping berdasarkan Peromo
Free Expedited Shipping for most orders over $49
→ Gratis jika pembelian di atas $49

Free Standard Shipping on Orders Over $49
→ Gratis jika pembelian di atas $49

Free Shipping on orders 35 and up
→ Gratis jika pembelian di atas $35

jika masih ada yang null setelah di filter 3 promo di atas akan di drop jika kurang dari 10% data yang null

setelah itu akan di transfrom menjadi int agar bisa di hitung

In [ ]:
def check_null( df):
    df_null = df.isnull().sum().reset_index()
    df_null.columns = ['col', 'missing_val']
    df_null_col = df_null[df_null['missing_val'] > 0].copy()
    df_null_col['missing_val'] = df_null_col['missing_val'].astype(int)
    df_null_col['missing_value_percentage'] = round((df_null_col['missing_val'] / len(df)) * 100,2)
    return df_null_col

In [ ]:
df_1 = check_null(df_sales)
df_1 = df_1[df_1['missing_value_percentage'] > 60]
col = list(df_1.col)
col

['ean',
 'Unnamed: 26',
 'Unnamed: 27',
 'Unnamed: 28',
 'Unnamed: 29',
 'Unnamed: 30']

In [ ]:
def drop_columns_null(df,col):
    df_clear = df.drop(columns=col)
    return df_clear

In [ ]:
df_dropped_null = drop_columns_null(df_sales, col)

In [ ]:
def fill_shipping(row, row_1, row_2):
    if pd.isna(row[row_1]) and row[row_2] > 45:
        return 'Free Expedited Shipping for most orders over $49'
    elif pd.isna(row[row_1]) and row[row_2] > 35:
        return 'Free Expedited Shipping for most orders over $35'
    else:
        return row[row_1]



In [ ]:

df_dropped_null['prices.shipping'] = df_dropped_null.apply(
    fill_shipping,
    axis=1,
    args=('prices.shipping', 'prices.amountMin')  
)

In [ ]:
check_null(df_dropped_null)

,col,missing_val,missing_value_percentage
9,prices.shipping,203,2.80
18,manufacturer,4014,55.37


In [ ]:
def fill_null(row):
    if pd.isna(row['manufacturer']):
        return row['brand']
    else:
        return row['manufacturer']

df_dropped_null['manufacturer'] = df_dropped_null.apply(fill_null, axis=1)


In [ ]:
df_dropped_null['manufacturer'] = df_dropped_null['manufacturer'].fillna(df_dropped_null['brand'])

In [ ]:
def drop_null_data(df):
    df_clean =  df.dropna()
    return df_clean

In [ ]:
check_null(df_dropped_null)

,col,missing_val,missing_value_percentage
9,prices.shipping,203,2.8


In [ ]:
df_clean = drop_null_data(df_dropped_null)

In [ ]:
check_null(df_clean)

,col,missing_val,missing_value_percentage


In [ ]:
def check_duplicated(df):
    df_cheked = df.duplicated().sum()
    return f'total data duplicate {df_cheked}'

In [ ]:
check_duplicated(df_clean)

'total data duplicate 0'

In [ ]:
# this is the clean data from sales_df
df_clean.head()

,id,prices.amountMax,prices.amountMin,prices.availability,prices.condition,prices.currency,prices.dateSeen,prices.isSale,prices.merchant,prices.shipping,...,dateUpdated,imageURLs,keys,manufacturer,manufacturerNumber,name,primaryCategories,sourceURLs,upc,weight
0,AVphzgbJLJeJML43fA0o,104.99,104.99,Yes,New,USD,"2017-03-30T06:00:00Z,2017-03-10T22:00:00Z,2017...",False,Bestbuy.com,Free Expedited Shipping for most orders over $49,...,2018-05-12T18:59:48Z,https://images-na.ssl-images-amazon.com/images...,sanusvlf410b110inchsuperslimfullmotionmountfor...,Sanus,VLF410B1,Sanus VLF410B1 10-Inch Super Slim Full-Motion ...,Electronics,https://www.amazon.com/Sanus-VLF410B1-10-Inch-...,7.94E+11,32.8 pounds
1,AVpgMuGwLJeJML43KY_c,69.00,64.99,In Stock,New,USD,2017-12-14T06:00:00Z,True,Walmart.com,Expedited,...,2018-06-13T19:39:02Z,https://images-na.ssl-images-amazon.com/images...,boytone2500w21chhometheatersystemblackdiamond/...,Boytone,BT-210F,Boytone - 2500W 2.1-Ch. Home Theater System - ...,Electronics,http://reviews.bestbuy.com/3545/4784804/review...,6.42E+11,14 pounds
2,AVpgMuGwLJeJML43KY_c,69.00,69.00,In Stock,New,USD,2017-09-08T05:00:00Z,False,Walmart.com,Expedited,...,2018-06-13T19:39:02Z,https://images-na.ssl-images-amazon.com/images...,boytone2500w21chhometheatersystemblackdiamond/...,Boytone,BT-210F,Boytone - 2500W 2.1-Ch. Home Theater System - ...,Electronics,http://reviews.bestbuy.com/3545/4784804/review...,6.42E+11,14 pounds
3,AVpgMuGwLJeJML43KY_c,69.99,69.99,Yes,New,USD,2017-10-10T05:00:00Z,False,Bestbuy.com,Free Expedited Shipping for most orders over $49,...,2018-06-13T19:39:02Z,https://images-na.ssl-images-amazon.com/images...,boytone2500w21chhometheatersystemblackdiamond/...,Boytone,BT-210F,Boytone - 2500W 2.1-Ch. Home Theater System - ...,Electronics,http://reviews.bestbuy.com/3545/4784804/review...,6.42E+11,14 pounds
4,AVpgMuGwLJeJML43KY_c,66.99,66.99,Yes,New,USD,2017-08-28T07:00:00Z,False,Bestbuy.com,Free Expedited Shipping for most orders over $49,...,2018-06-13T19:39:02Z,https://images-na.ssl-images-amazon.com/images...,boytone2500w21chhometheatersystemblackdiamond/...,Boytone,BT-210F,Boytone - 2500W 2.1-Ch. Home Theater System - ...,Electronics,http://reviews.bestbuy.com/3545/4784804/review...,6.42E+11,14 pounds


In [ ]:
def create_shipping_to_float(col):
    row_int = col
    if 'USD' in row_int:
        val_float = row_int.split(' ')[1]
        return float(val_float)
    else:
        return 0.0

In [ ]:
df_clean['shipping_cost'] = df_clean['prices.shipping'].apply(create_shipping_to_float)

/tmp/ipykernel_17412/232550742.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['shipping_cost'] = df_clean['prices.shipping'].apply(create_shipping_to_float)


In [ ]:
df_clean['shipping_cost'].value_counts()

shipping_cost
0.00      6980
6.65         3
196.92       2
10.10        2
10.00        2
5.00         2
150.00       2
2.99         2
14.00        2
7.95         1
60.00        1
0.99         1
35.03        1
15.68        1
20.00        1
13.32        1
5.99         1
22.64        1
13.25        1
12.78        1
2.85         1
25.00        1
15.42        1
7.25         1
26.09        1
11.30        1
20.55        1
13.95        1
75.00        1
7.81         1
27.94        1
25.76        1
13.60        1
18.60        1
13.61        1
3.99         1
7.50         1
12.21        1
3.39         1
22.13        1
6.00         1
11.75        1
4.88         1
14.99        1
18.00        1
4.96         1
179.35       1
24.04        1
71.77        1
10.95        1
140.00       1
29.99        1
23.47        1
79.42        1
37.65        1
16.45        1
3.80         1
16.00        1
Name: count, dtype: int64

In [ ]:
['prices.shipping'].value_counts()

prices.shipping
Free Expedited Shipping for most orders over $49    4124
FREE                                                 737
Value                                                697
Standard                                             653
Expedited                                            411
                                                    ... 
USD 37.65                                              1
Free Next Day Delivery (USA)                           1
USD 16.45                                              1
USD 3.80                                               1
USD 16.00                                              1
Name: count, Length: 75, dtype: int64

In [ ]:
df_clean.columns

Index(['id', 'prices.amountMax', 'prices.amountMin', 'prices.availability',
       'prices.condition', 'prices.currency', 'prices.dateSeen',
       'prices.isSale', 'prices.merchant', 'prices.shipping',
       'prices.sourceURLs', 'asins', 'brand', 'categories', 'dateAdded',
       'dateUpdated', 'imageURLs', 'keys', 'manufacturer',
       'manufacturerNumber', 'name', 'primaryCategories', 'sourceURLs', 'upc',
       'weight', 'shipping_cost'],
      dtype='object')

In [ ]:
df_clean

In [18]:
pip install python-dotenv


Note: you may need to restart the kernel to use updated packages.


In [17]:
pip install os

ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os
Note: you may need to restart the kernel to use updated packages.
